# Data Loading Methods in PyTorch


The goal of this notebook is to explore PyTorch's tools for lazily loading datasets from disk instead of storing the datasets in memory. During supervised training over large parameter spaces, it'll be important to store many ground states for many parameter configurations before computation, as calculating them on the fly becomes time-consuming.

The goal is a tool living in the CPU that loads relevant data from disk (possibly without order, like random-access memory) to feed it to the model on the GPU. **As the model is training on that data on the GPU, the CPU should fetch the next batch of data.**


In [28]:
import torch
import numpy as np
import itertools
from Hamiltonian import Ising, XYZ
from optimizer_supervised import Optimizer
from model import TransformerModel

In [8]:
ham = Ising(3)
energy, psi = ham.calc_ground(param=torch.tensor([1.3]))
print(energy)
print(psi)

-4.657965112114927
tensor([ 0.5694, -0.2421, -0.2421,  0.2421, -0.2421,  0.2421,  0.2421, -0.5694],
       dtype=torch.float64)


/home/spandan/Projects/tqs/Hamiltonian_utils.py:44: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  system_size = torch.tensor(system_size, dtype=torch.int64).reshape(-1)


In [15]:
ham = XYZ(5)
energy, psi = ham.calc_ground(param=torch.tensor([0.5, 0, 0.2]))
print(energy)
print(psi)

-5.762087103566636
tensor([-3.2893e-17-6.2905e-17j, -3.3812e-02-5.9542e-03j,
        -1.6316e-02+1.1113e-04j, -1.4513e-17+8.3671e-17j,
         2.3728e-02+6.0229e-03j,  3.0557e-17+5.2746e-17j,
         2.1343e-17+5.9191e-18j,  1.5502e-01-1.0559e-03j,
         3.0981e-02+3.6112e-03j, -7.7125e-17-2.1299e-16j,
         5.0608e-17+2.3897e-17j, -4.7628e-01-5.5517e-02j,
        -3.0463e-17+8.7971e-17j,  5.1980e-01+9.1536e-02j,
        -2.2544e-01-5.7225e-02j,  1.8628e-16+3.3999e-17j,
        -4.5807e-03-3.7911e-03j, -3.1454e-17+1.5063e-16j,
         4.9184e-17-8.0072e-17j,  3.2125e-01+5.6573e-02j,
        -4.1779e-17-5.3120e-17j, -3.6477e-01-9.2592e-02j,
         7.0420e-02+5.8281e-02j,  1.6733e-17-3.7297e-17j,
         6.2260e-17+1.4176e-16j,  4.3522e-02+3.6020e-02j,
         2.5083e-01-1.7084e-03j, -1.9619e-16+1.7983e-17j,
        -2.9435e-01-3.4311e-02j,  2.5486e-16-2.8029e-17j,
        -3.3375e-16+6.9043e-18j, -1.1120e-17+4.7325e-18j],
       dtype=torch.complex128)


/home/spandan/Projects/tqs/Hamiltonian_utils.py:44: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  system_size = torch.tensor(system_size, dtype=torch.int64).reshape(-1)


The following functions are from optimization_tests.ipynb (or optimizer_supervised.py):


In [42]:
def generate_parameter_range(start, end, step):
    """
    A simple generator returning the next value in a range of values
    whenever called, according to a step size.
    """
    value = start
    while value < end:
        yield value
        value += step


def generate_parameter_points(parameter_ranges, step_sizes, distribution=None):
    """
    Generate all possible combinations of parameter values for a model
    (i.e., the Cartesian product of values of parameters in a slice of parameter space)

    Parameters:
        parameter_ranges: torch.Tensor of shape (2, n_parameters)
            The starting and ending values for each dimension of the slice of parameter space
        step_sizes: torch.Tensor of shape (n_parameters,)
            The step size for each dimension of the slice of parameter space
        distribution: N/A
            TODO: Not implemented

    """

    parameter_ranges = parameter_ranges.T  # TODO: remove

    if distribution is not None:
        raise NotImplementedError(
            "Sampling using a custom distribution is not implemented yet."
        )

    # Every possible individual parameter value for each parameter, in order
    parameter_ranges = [
        generate_parameter_range(start.item(), end.item(), step.item())
        for (start, end), step in zip(parameter_ranges, step_sizes)
    ]

    return itertools.product(*parameter_ranges)

In [46]:
param_ranges = torch.tensor([[0, -1, 0.2], [1, 1, 0.3]])
param_steps = torch.tensor([0.1, 0.1, 0.1])

print(param_ranges)
print(param_steps)

tensor([[ 0.0000, -1.0000,  0.2000],
        [ 1.0000,  1.0000,  0.3000]])
tensor([0.1000, 0.1000, 0.1000])


In [47]:
points_generator = generate_parameter_points(param_ranges, param_steps)

In [50]:
for point in points_generator:
    print(torch.tensor(point))

tensor([ 0.0000, -1.0000,  0.2000])
tensor([ 0.0000, -1.0000,  0.3000])
tensor([ 0.0000, -0.9000,  0.2000])
tensor([ 0.0000, -0.9000,  0.3000])
tensor([ 0.0000, -0.8000,  0.2000])
tensor([ 0.0000, -0.8000,  0.3000])
tensor([ 0.0000, -0.7000,  0.2000])
tensor([ 0.0000, -0.7000,  0.3000])
tensor([ 0.0000, -0.6000,  0.2000])
tensor([ 0.0000, -0.6000,  0.3000])
tensor([ 0.0000, -0.5000,  0.2000])
tensor([ 0.0000, -0.5000,  0.3000])
tensor([ 0.0000, -0.4000,  0.2000])
tensor([ 0.0000, -0.4000,  0.3000])
tensor([ 0.0000, -0.3000,  0.2000])
tensor([ 0.0000, -0.3000,  0.3000])
tensor([ 0.0000, -0.2000,  0.2000])
tensor([ 0.0000, -0.2000,  0.3000])
tensor([ 0.0000, -0.1000,  0.2000])
tensor([ 0.0000, -0.1000,  0.3000])
tensor([0.0000e+00, 1.4901e-08, 2.0000e-01])
tensor([0.0000e+00, 1.4901e-08, 3.0000e-01])
tensor([0.0000, 0.1000, 0.2000])
tensor([0.0000, 0.1000, 0.3000])
tensor([0.0000, 0.2000, 0.2000])
tensor([0.0000, 0.2000, 0.3000])
tensor([0.0000, 0.3000, 0.2000])
tensor([0.0000, 0.3000, 0

In [49]:
points_generator = generate_parameter_points(param_ranges, param_steps)